In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd 
import random
import os  
from selenium.webdriver.common.keys import Keys

In [2]:
#1. 사용자 인터페이스 설정

print('='*100)
print('수비를 위한 네이버 웹소설 댓글 및 평점 크롤러')
print('-'*100)
print('이 크롤러는 1화부터 등록된 마지막 화 까지 댓글 및 평점을 순차 수집합니다')
print('='*100)
nov_title = input('1. 댓글 및 평점 정보 수집하고 싶은 웹소설 제목을 입력하세요 : ')

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

f_dir = input('''2. 크롤링 결과물 파일 저장할 문서 경로를 폴더 이름까지만 입력해주세요(예 : /Users/kibeomkim/Desktop/folder2) : 
모르겠으면 엔터 키를 눌러주세요. 기본 경로로 저장됩니다. ''')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터'
os.makedirs(f_dir+'/'+s+'-'+nov_title)
os.chdir(f_dir+'/'+s+'-'+nov_title)

#2. 파일 확장자 & 경로 미리 변수로 담아두기 
f_name = s+'-'+nov_title+'.txt'

fc_name = s+'-'+nov_title+'.csv'
fc_name2 = s+'-'+nov_title+'댓글정보'+'.csv'

fx_name = s+'-'+nov_title+'.xlsx'
fx_name2 = s+'-'+nov_title+'댓글정보'+'.xlsx'
s_time = time.time()

#3. 웹드라이버 호출 & 네이버 웹소설 페이지로 이동하기
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('disable-gpu')
options.add_argument('lang=ko_KR')
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

driver = webdriver.Chrome(path, options=options)
driver.get('https://novel.naver.com/webnovel/weekday.nhn')
time.sleep(random.randrange(2, 5))


#4. 웹소설 검색창 찾고 검색어 넣기 + 엔터
element1 = driver.find_element_by_xpath('/html/body/div/div[2]/div/div[1]/div[2]/span/input')
element1.send_keys(nov_title)
element1.send_keys('\n')
time.sleep(random.randrange(3, 5))


url_list = []
no0 = 0
#5. 검색어로 조회된 웹소설 모두 보여주기 
html0 = driver.page_source 
soup0 = BeautifulSoup(html0, 'html.parser')

try : 
    li_list = soup0.find('div', 'srch_cont NE=a:all').find_all('li')
    number = len(li_list)
    print('%s 키워드로 총 %s 개 웹소설이 발견되었습니다.'%(nov_title, number))
except : 
    print('%s 키워드로 검색된 웹소설이 없습니다.')
    driver.quit()
    
for a in li_list : 
    each_title = a.find('span', 'ellipsis')
    real_title = each_title.get_text().replace('\n', "").strip()
    print('%s. %s' %(no0, real_title))
    url = a.find('a')['href']
    url_list.append(url)
    no0 += 1
try : 
    choice = int(input('0번~%s 번 중 댓글 및 평점 수집할 웹소설 번호를 하나만 입력하세요 : '))
except : 
    print('번호를 하나만 입력하세요')


driver.quit()

path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
driver = webdriver.Chrome(path, options=options)
driver.get('https://novel.naver.com'+url_list[choice])
time.sleep(random.randrange(2, 5))



수비를 위한 네이버 웹소설 댓글 및 평점 크롤러
----------------------------------------------------------------------------------------------------
이 크롤러는 1화부터 등록된 마지막 화 까지 댓글 및 평점을 순차 수집합니다
1. 댓글 및 평점 정보 수집하고 싶은 웹소설 제목을 입력하세요 : 실베스터 공녀
2. 크롤링 결과물 파일 저장할 문서 경로를 폴더 이름까지만 입력해주세요(예 : /Users/kibeomkim/Desktop/folder2) : 
모르겠으면 엔터 키를 눌러주세요. 기본 경로로 저장됩니다. 
실베스터 공녀 키워드로 총 1 개 웹소설이 발견되었습니다.
0. 황태자의 보좌관 실베스터 공녀
0번~%s 번 중 댓글 및 평점 수집할 웹소설 번호를 하나만 입력하세요 : 0


In [3]:
html44 = driver.page_source
soup44 = BeautifulSoup(html44, 'html.parser')
text44 = soup44.find('h3', 'tit_num_list').find('span','total').get_text()
text45 = int(text44.replace('(', "").replace(")", "").strip())
print('총 %s 회차가 존재합니다' %text45)

총 88 회차가 존재합니다


In [4]:
#1화부터 보기 클릭
try : 
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div[2]/div[1]/h3/span[3]/label[2]').click()
    time.sleep(3)
except : 
    print("")

In [5]:
#7. 평점 수집하기 
star_grade1 = []
star_people1 = []
heart1 = []
nicknamelist1 = []
contentslist1 = []
datelist1 = []
gogamlist1 = []
bgogamlist1 = []
h1 = []
h3 = []

no22 = 2
f = open(f_name, 'a', encoding='utf-8')
no11 = 1
no33 = 1
no45 = 1
no55 = 1
no66 = 0

while no22 == 2 : 
    html23 = driver.page_source 
    soup23 = BeautifulSoup(html23, 'html.parser')
    li_number = len(soup23.find('ul').find_all('li'))
    
    for x in range(1, 11) : 
        driver.find_element_by_xpath('/html/body/div/div[3]/div/div[2]/div[1]/ul/li[%s]/a/div/p[1]' %x).click()
        time.sleep(2)
        
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(1)
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_UP)
        time.sleep(3)

        html1 = driver.page_source 
        soup1 = BeautifulSoup(html1, 'html.parser')
        print('%s화 평점 및 댓글 수집합니다 ==============' %no11)
        f.write('%s화 평점 및 댓글 수집합니다 ===========' %no11 + '\n')
        h2 = '%s화' %no11
        h1.append(h2)
    
        star_grade = soup1.find('div', 'section_grade').find('em', id='currentStarScore').get_text().replace('\n', "").strip()
        star_grade1.append(star_grade)
        print('1.별점:',star_grade)
        f.write('1.별점:'+star_grade+'\n')
        time.sleep(1)
    
        star_people = soup1.find('div', 'section_grade').find('span', 'num').get_text().strip()
        star_people1.append(star_people)
        print('2.별점 준 사람 수:',star_people)
        f.write('2.별점 준 사람 수:'+star_people+'\n')
        time.sleep(1)
    
        heart = soup1.find('div', 'section_function_viewer').find('em').get_text().strip()
        heart1.append(heart)
        print('3.하트 누른 사람 수:',heart)
        print('-'*100)
        f.write('3.하트 누른 사람 수:'+heart+'\n')
        f.write('-'*100+'\n')
        time.sleep(1)
        
        driver.switch_to.frame('nCommentIframe')
        html88 = driver.page_source
        soup88 = BeautifulSoup(html88, 'html.parser')
        ddat_number = int(soup88.find('span', 'u_cbox_count').get_text().strip())
        print('총 %s개 댓글이 있습니다' %ddat_number)
        
        
    
        #8. 댓글 수집하기 
        while no22 == 2 : 
            no66 += 1 
            html3 = driver.page_source
            soup3 = BeautifulSoup(html3, 'html.parser')
            cbox_list = soup3.find('ul', 'u_cbox_list').find_all('div', 'u_cbox_area')
            print(len(cbox_list))
            time.sleep(3)
        
            for b in cbox_list : 
                h3.append(h2)
                nick_name = b.find('div', 'u_cbox_info').get_text().replace('\n', "").strip()
                nicknamelist1.append(nick_name)
                print('4.댓글 작성자 닉네임 :',nick_name)
                f.write('4.댓글 작성자 닉네임 :'+nick_name+'\n')
                #time.sleep(1)
        
                contents = b.find('div', 'u_cbox_text_wrap').find('span', 'u_cbox_contents').get_text().strip()
                contentslist1.append(contents)
                print('5.댓글 내용 :',contents)
                f.write('5.댓글 내용 :'+contents+'\n')
                #time.sleep(1)
        
                date = b.find('div', 'u_cbox_info_base').find('span', 'u_cbox_date').get_text().replace('\n', "").strip()
                datelist1.append(date)
                print('6.댓글 작성 날짜 :',date)
                f.write('6.댓글 작성 날짜 :'+date+'\n')
                #time.sleep(1)
        
                gogam = b.find('em', 'u_cbox_cnt_recomm').get_text().strip()
                gogamlist1.append(gogam)
                print('7.공감횟수 :',gogam)
                f.write('7.공감횟수 :'+gogam+'\n')
                #time.sleep(1)
        
                bgogam = b.find('em', 'u_cbox_cnt_unrecomm').get_text().strip()
                bgogamlist1.append(bgogam)
                print('8.비공감횟수 :',bgogam)
                print('\n')
                f.write('8.비공감횟수 :'+bgogam+'\n')
                f.write('\n')
                #time.sleep(1)
                

                    
            try : 
                if no55 == 1 : 
                    driver.find_element_by_xpath('/html/body/div/div/div[5]/div/a[%s]'%no55).click()
                    no55 = no55 + 2
                    time.sleep(2)
                elif no55 != 1 : 
                    if no66 == 11 : 
                        no55 = 3
                    if no55 == 13 : 
                        no55 = 3
                    driver.find_element_by_xpath('/html/body/div/div/div[5]/div/a[%s]' %no55).click()
                    no55 += 1
                    time.sleep(2)
            except : 
                print('조회 가능한 모든 댓글을 조회했습니다')
                print('\n')
                f.write('조회 가능한 모든 댓글을 조회했습니다'+'\n')
                break
                
        print('%s화 평점 및 댓글 수집 완료 ========' %no11)
        print('\n')
        f.write('%s화 평점 및 댓글 수집 완료 =======' %no11+'\n')
        driver.back()
        no55 = 1
        no66 = 0
        time.sleep(4)
        no11 += 1
        if no11 > text45 : 
            break
    try : 
        driver.find_element_by_xpath('/html/body/div/div[3]/div/div[2]/div[1]/div[3]/a[%s]' %no33).click()
        no33 += 1
    except : 
        print('가능한 모든 회차를 조회하였습니다')
        break

f.close()
driver.quit()
e_time = time.time()
total_time = e_time - s_time

ddubi = pd.DataFrame()
ddub2 = pd.DataFrame()

ddub2['회차정보'] = h3
ddub2['댓글 작성자 닉네임'] = nicknamelist1
ddub2['댓글 내용'] = contentslist1
ddub2['댓글 작성 날짜'] = datelist1
ddub2['공감횟수'] = gogamlist1
ddub2['비공감횟수'] = bgogamlist1


ddubi['회차'] = pd.Series(h1)
ddubi['별점'] = pd.Series(star_grade1)
ddubi['별점 준 사람 수'] = pd.Series(star_people1)
ddubi['하트 누른 사람 수'] = pd.Series(heart1)


ddub2.to_csv(fc_name2, encoding='utf-8-sig')
ddub2.to_excel(fx_name2)

ddubi.to_csv(fc_name, encoding='utf-8-sig')
ddubi.to_excel(fx_name)

print('='*100)
print('%s 웹소설, 총 %s 회차의 댓글 및 평점을 수집했습니다' %(nov_title, no11))
print('총 소요시간은 %s 초 입니다' %round(total_time, 1))
print('='*100)



1화 평점 및 댓글 수집합니다 ==============
1.별점: 10.0
2.별점 준 사람 수: 50명
3.하트 누른 사람 수: 56
----------------------------------------------------------------------------------------------------
총 4개 댓글이 있습니다
4
4.댓글 작성자 닉네임 : 지아(jia7****)
5.댓글 내용 : 없었데 - 없었대  입니당 글은 재밌네용
6.댓글 작성 날짜 : 2021-01-17 10:22
7.공감횟수 : 0
8.비공감횟수 : 0


4.댓글 작성자 닉네임 : 은하계(joel****)
5.댓글 내용 : 정주행 시작!
6.댓글 작성 날짜 : 2020-11-18 16:49
7.공감횟수 : 0
8.비공감횟수 : 1


4.댓글 작성자 닉네임 : gran****(gran****)
5.댓글 내용 : 즐독하고 갑니다
6.댓글 작성 날짜 : 2020-11-15 19:40
7.공감횟수 : 2
8.비공감횟수 : 0


4.댓글 작성자 닉네임 : momk****(momk****)
5.댓글 내용 : 재밌어요 ㅎㅎㅎ
6.댓글 작성 날짜 : 2020-07-23 18:01
7.공감횟수 : 4
8.비공감횟수 : 0


조회 가능한 모든 댓글을 조회했습니다


1화 평점 및 댓글 수집 완료 ========


2화 평점 및 댓글 수집합니다 ==============
1.별점: 10.0
2.별점 준 사람 수: 41명
3.하트 누른 사람 수: 44
----------------------------------------------------------------------------------------------------
총 1개 댓글이 있습니다
1
4.댓글 작성자 닉네임 : 은하계(joel****)
5.댓글 내용 : 누가 감히 여주를 건드리려해?!
6.댓글 작성 날짜 : 2020-11-19 20:16
7.공감횟수 : 1
8.비공감횟수 : 0


조회 가능한 모든 댓글을 조

총 1개 댓글이 있습니다
1
4.댓글 작성자 닉네임 : 개구리rornfr88(wkda****)
5.댓글 내용 : 
6.댓글 작성 날짜 : 2020-12-08 00:16
7.공감횟수 : 0
8.비공감횟수 : 0


조회 가능한 모든 댓글을 조회했습니다


23화 평점 및 댓글 수집 완료 ========


24화 평점 및 댓글 수집합니다 ==============
1.별점: 10.0
2.별점 준 사람 수: 18명
3.하트 누른 사람 수: 23
----------------------------------------------------------------------------------------------------
총 1개 댓글이 있습니다
1
4.댓글 작성자 닉네임 : 개구리rornfr88(wkda****)
5.댓글 내용 : 죽고싶어서 용을 쓰는군요
6.댓글 작성 날짜 : 2020-12-08 00:26
7.공감횟수 : 0
8.비공감횟수 : 0


조회 가능한 모든 댓글을 조회했습니다


24화 평점 및 댓글 수집 완료 ========


25화 평점 및 댓글 수집합니다 ==============
1.별점: 10.0
2.별점 준 사람 수: 18명
3.하트 누른 사람 수: 22
----------------------------------------------------------------------------------------------------
총 0개 댓글이 있습니다
0
조회 가능한 모든 댓글을 조회했습니다


25화 평점 및 댓글 수집 완료 ========


26화 평점 및 댓글 수집합니다 ==============
1.별점: 10.0
2.별점 준 사람 수: 19명
3.하트 누른 사람 수: 21
----------------------------------------------------------------------------------------------------
총 0개 댓글이 있습니다
0
조회 가능한 모든 댓글을 조회했습니다


2

48화 평점 및 댓글 수집합니다 ==============
1.별점: 9.44
2.별점 준 사람 수: 16명
3.하트 누른 사람 수: 19
----------------------------------------------------------------------------------------------------
총 2개 댓글이 있습니다
2
4.댓글 작성자 닉네임 : AMOX(5131****)
5.댓글 내용 : 아오 여주 성격 빡쳐
6.댓글 작성 날짜 : 2020-11-18 15:54
7.공감횟수 : 1
8.비공감횟수 : 0


4.댓글 작성자 닉네임 : 애플민트(pis3****)
5.댓글 내용 : 저런 여자 싫다. 진심과 가식도 구분 못하고 잘생긴 남자의 애절함에 넘어가는 바보같음. 그러다가 에밀리에게 속아서 죽을뻔 했으면서 진짜 사람 볼줄 모르네.
6.댓글 작성 날짜 : 2020-11-13 21:48
7.공감횟수 : 3
8.비공감횟수 : 1


조회 가능한 모든 댓글을 조회했습니다


48화 평점 및 댓글 수집 완료 ========


49화 평점 및 댓글 수집합니다 ==============
1.별점: 9.47
2.별점 준 사람 수: 17명
3.하트 누른 사람 수: 20
----------------------------------------------------------------------------------------------------
총 2개 댓글이 있습니다
2
4.댓글 작성자 닉네임 : 모카커피(ey11****)
5.댓글 내용 : 능력여주 능력남주라는데.노려보고, 째려보고, 소리치고, 살기뿌리기. 외에는 할 줄 아는게 없고. 자기 맹신에 멍청하기까지.캐릭터들의 매력이 전혀 없습니다. 악역은 무식 무능의 절정이구요.솔직히 저는 글을 쓰는 사람도 아니고, 쓴다고 할지라도 이 정도도 못 쓸 사람입니다.하지만 읽는 건 할 수 있습니다.이건 비판이 아닙니다. 작가님의 필력이 더 좋아지시길 바라는 마음에 몇 자 적었습니다.
6.댓글 작성 날짜 

총 2개 댓글이 있습니다
2
4.댓글 작성자 닉네임 : 까만콩(ab20****)
5.댓글 내용 : 후작과 에밀리 가라
6.댓글 작성 날짜 : 2020-11-24 06:23
7.공감횟수 : 2
8.비공감횟수 : 0


4.댓글 작성자 닉네임 : 꿈꾸는 아이(wnwn****)
5.댓글 내용 : 진짜.. 이 소설의 악당들 빨리 없어져라..그리고 후작가 쌤통이다ㅋㅋ
6.댓글 작성 날짜 : 2020-11-24 01:23
7.공감횟수 : 3
8.비공감횟수 : 0


조회 가능한 모든 댓글을 조회했습니다


63화 평점 및 댓글 수집 완료 ========


64화 평점 및 댓글 수집합니다 ==============
1.별점: 10.0
2.별점 준 사람 수: 14명
3.하트 누른 사람 수: 18
----------------------------------------------------------------------------------------------------
총 3개 댓글이 있습니다
3
4.댓글 작성자 닉네임 : 까만콩(ab20****)
5.댓글 내용 : 사건사고가 많네
6.댓글 작성 날짜 : 2020-11-28 11:23
7.공감횟수 : 0
8.비공감횟수 : 0


4.댓글 작성자 닉네임 : 엔젤(cko2****)
5.댓글 내용 : 잘 읽었습니다   정령들이  열심 활약하고 있네여
6.댓글 작성 날짜 : 2020-11-28 09:09
7.공감횟수 : 1
8.비공감횟수 : 0


4.댓글 작성자 닉네임 : 꿈꾸는 아이(wnwn****)
5.댓글 내용 : 아니...왜이리 평화가 안찾아와ㅠㅠ 하나 해결하며 또다시 터지고ㅠㅠ 하.. 진짜 화가난다 화가
6.댓글 작성 날짜 : 2020-11-27 23:36
7.공감횟수 : 1
8.비공감횟수 : 0


조회 가능한 모든 댓글을 조회했습니다


64화 평점 및 댓글 수집 완료 ========


65화 평점 및 댓글 수집합니다 ==============
1.별점: 10.0
2.별점 준 사람 수: 11명
3.하트 누른 사

85화 평점 및 댓글 수집합니다 ==============
1.별점: 10.0
2.별점 준 사람 수: 9명
3.하트 누른 사람 수: 11
----------------------------------------------------------------------------------------------------
총 1개 댓글이 있습니다
1
4.댓글 작성자 닉네임 : 우아한걸(rose****)
5.댓글 내용 : 연참은.. 사랑입니다❤️
6.댓글 작성 날짜 : 2021-01-22 20:53
7.공감횟수 : 1
8.비공감횟수 : 0


조회 가능한 모든 댓글을 조회했습니다


85화 평점 및 댓글 수집 완료 ========


86화 평점 및 댓글 수집합니다 ==============
1.별점: 10.0
2.별점 준 사람 수: 12명
3.하트 누른 사람 수: 14
----------------------------------------------------------------------------------------------------
총 3개 댓글이 있습니다
3
4.댓글 작성자 닉네임 : 푼제리(hi42****)
5.댓글 내용 : 음..? 여러분 누가 후작인가요? 제 눈엔 그저 사형감인 죄인으로 밖에 안보이는데..아 제 눈도 이상하군요 사형이라니 더 심한 벌을 받아야겠죠? 발을 잘라서 평민으로 살게할까...아님 손목을 잘라서 성격드러운 귀족의 하인이되게할까...뭐가 좋아요 작가님?
6.댓글 작성 날짜 : 2021-01-22 21:07
7.공감횟수 : 2
8.비공감횟수 : 0


4.댓글 작성자 닉네임 : 우아한걸(rose****)
5.댓글 내용 : 후작..멀리 안나갑니다살펴가세요ㅎㅎㅎ
6.댓글 작성 날짜 : 2021-01-22 20:57
7.공감횟수 : 2
8.비공감횟수 : 0


4.댓글 작성자 닉네임 : kukil(kuki****)
5.댓글 내용 : 다음화 부탁 드려요
6.댓글 작성 날짜 : 2021-01-22 20:20
7.공감횟수 : 4
8.비공

list88 = []
for d in list88 : 
    print(d) # for 반복문을 [] 빈 리스트에서 돌리면 실행은 된다. 근데 아무 결과값도 내놓지 않는다.왜? 
print('a') #--> for '반복문'이 아니라 for '조건문'이다. 
# --> for d in list88 매칭할 조건이 안 되니까,
# --> for 코드 블록을 통째로 그냥 무시한다.  그래서 아무 결과값도 내놓지 않는거다. 
# while도 엄밀히 말하면 조건문이다. 